## **Para o projeto**

In [ ]:
!pip install gspread
!pip install -U gspread oauth2client
!pip install fsspec
import pandas as pd
import gspread
import json
import ssl #para lidar com o certificado de segurança
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
nome = "xxxx.json"
spreadsheet_id = "xxxxxx"
arquivo = open(nome)
credentials = json.load(arquivo)
service_account = gspread.service_account_from_dict(credentials) # autenticação

In [3]:
amazonia = ['AC', 'AM', 'AP','MA','PA','MT','RR','RO','TO']

In [4]:
def get_multas_amazonia_legal():  
    UF = input("Digite a sigla da unidade de federação: ")
    UF = UF.upper()
    if UF in amazonia:
      url = "https://dadosabertos.ibama.gov.br/dados/SICAFI/"+UF+"/Quantidade/multasDistribuidasBensTutelados.csv"
      df = pd.read_csv(url, sep=';')
      spreadsheet = service_account.open_by_key(spreadsheet_id) # "abrir" o arquivo
      worksheet = spreadsheet.worksheet(UF) # escolhe uma aba
      df = df.loc[df['Situação Débito'] == 'Para homologação/prazo de defesa']
      df = df.loc[df['Tipo Auto'] == 'Multa']
      # deletando colunas que não são necessárias na visualização
      del df['Nº AI']
      del df['Data Auto']
      del df['Enquadramento Legal']
      dados = df.groupby(['Município', 'Tipo Infração', 'Última Atualização Relatório']).size().to_frame(name = 'count').reset_index().sort_values(by=['count'],ascending=False)
      worksheet.update([dados.columns.values.tolist()] + dados.values.tolist())
    else:
      print("Esta UF não pertence ao território da Amazônia Legal!")
    return UF

In [5]:
get_multas_amazonia_legal()

Digite a sigla da unidade de federação: sp
Esta UF não pertence ao território da Amazônia Legal!


'SP'

In [6]:
get_multas_amazonia_legal()

Digite a sigla da unidade de federação: rr


'RR'

# **Anotações - funcionamento**

In [7]:
df = pd.read_csv("https://dadosabertos.ibama.gov.br/dados/SICAFI/RR/Quantidade/multasDistribuidasBensTutelados.csv", sep=';')
df = df.loc[df['Situação Débito'] == 'Para homologação/prazo de defesa']
df = df.loc[df['Tipo Auto'] == 'Multa']
# deletando colunas que não são necessárias na visualização
del df['Nº AI']
del df['Data Auto']
del df['Enquadramento Legal']
dados = df.groupby(['Município', 'Tipo Infração', 'Última Atualização Relatório']).size().to_frame(name = 'count').reset_index().sort_values(by=['count'],ascending=False)

In [8]:
dados

,Município,Tipo Infração,Última Atualização Relatório,count
52,RORAINOPOLIS,Flora,13/01/2022 19:10,216
25,CARACARAI,Flora,13/01/2022 19:10,90
11,BOA VISTA,Flora,13/01/2022 19:10,63
8,BOA VISTA,Cadastro Técnico Federal,13/01/2022 19:10,60
39,MUCAJAI,Flora,13/01/2022 19:10,56
30,CAROEBE,Flora,13/01/2022 19:10,45
9,BOA VISTA,Controle ambiental,13/01/2022 19:10,43
14,BOA VISTA,Outras,13/01/2022 19:10,33
20,CANTA,Flora,13/01/2022 19:10,26
49,RORAINOPOLIS,Cadastro Técnico Federal,13/01/2022 19:10,22
